In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter
from typing import Generator, Tuple

In [2]:
def load_file(path) -> pd.DataFrame:
    """Загружает данные из exel файла, удаляет незаполненные значения, формирует колонку временной метки"""
    
    df = pd.read_excel(path, header=0)
    df['DateTime']= pd.to_datetime(df['Date'].astype(str)+' '+ df['Time'].astype(str))
    df = df.drop(['Date', 'Time'], axis=1)
    df.replace('-',np.NaN, inplace=True)
    df.dropna(axis=0, how='any', inplace=True)
    return df

def remove_outliers_with_datetime(df: pd.DataFrame, threshold_multiplier: float) -> pd.DataFrame:
    # Копируем DataFrame, чтобы сохранить оригинальные данные
    filtered_df = df.copy()
    # Список для сохранения индексов строк, которые не являются выбросами
    valid_indices = []

    # Проходим по всем столбцам, кроме последнего (DateTime)
    for column in df.columns[:-1]:
        # Вычисление среднего и стандартного отклонения
        mean = df[column].mean()
        std = df[column].std()

        # Определение границ для выбросов
        upper_threshold = mean + threshold_multiplier * std
        lower_threshold = mean - threshold_multiplier * std

        # Находим валидные строки, которые не являются выбросами
        valid_mask = (df[column] <= upper_threshold) & (df[column] >= lower_threshold)
        
        # Сохраняем индексы валидных строк для текущего столбца
        if not valid_indices:
            # Если список индексов еще пуст, инициализируем его
            valid_indices = df.index[valid_mask].tolist()
        else:
            # Объединяем индексы с уже существующими, оставляя только те, которые валидны во всех колонках
            valid_indices = [index for index in valid_indices if index in df.index[valid_mask]]

    # Отфильтровываем DataFrame, оставляя только строки с валидными индексами
    filtered_df = df.loc[valid_indices].reset_index(drop=True)

    return filtered_df


In [12]:
df_X1.shape

(11570, 88)

In [3]:
df_X1 = remove_outliers_with_datetime(load_file('Исходные данные/Процессные данные Давление насыщенных паров в продукте, зимний период.xlsx'),3)
df_X2 = remove_outliers_with_datetime(load_file('Исходные данные/Процессные данные Конец кипения легкого бензина.xlsx'),3)
df_X3 = remove_outliers_with_datetime(load_file('Исходные данные/Процессные данные Содержание олефинов в продукте.xlsx'),3)

In [4]:
pd.set_option('display.max_columns', None)
min_var = 0.01

In [5]:
def dataframe_info(df):
    df = df.drop(columns=['DateTime'])
    variance = df.var()
    std = df.std()
    means = df.mean()
    medians = df.median()
    mins = df.min()
    maxes = df.max()
    df_info = pd.DataFrame([variance, std, means, medians, mins, maxes], index=['variance', 'standart deviation', 'mean', 'median', 'min', 'max'])
    return df_info

def drop_low_variance(df, df_info, min_var):
    # Получение названий столбцов до удаления
    original_cols = set(df.columns)
    
    # Создание маски булевых значений, которое будет True для столбцов с дисперсией
    # меньше, чем заданный порог (min_var), и False для остальных
    m1 = df_info.loc['variance'] < min_var
    
    # Выбор имен столбцов из исходного DataFrame, которые соответствуют условию маски
    drop_cols = df.columns[:-1][m1]
    
    # Удаление столбцов с низкой дисперсией из исходного DataFrame
    df.drop(columns=list(drop_cols), inplace=True)
    
    # Получение названий столбцов после удаления
    remaining_cols = set(df.columns)
    
    # Определение разницы между двумя наборами названий столбцов, чтобы увидеть, какие были удалены
    removed_cols = list(original_cols - remaining_cols)
    
    # Возвращаем исходный DataFrame (уже измененный) и список удаленных столбцов
    return removed_cols

In [6]:
df_info_X1 = dataframe_info(df_X1)


In [7]:
removed_cols_1 = drop_low_variance(df_X1, df_info_X1, min_var)
print("Удаленные столбцы:", removed_cols_1)

Удаленные столбцы: ['Давление С-1201', 'Давление в емкости V-1201', 'Возврат ТБ в V-1205 после насосов Р-1203', 'Давление ТБ перед Е-1101', 'Давление верха С-1201', 'Перепад давления реактора R-1201', 'Давление на входе в R-1101', 'Давление V-1203', 'Давление в емкости V-1101', 'Перепад давления реактора R-1101', 'Расход пара перед Е-1103']


In [8]:
df_info_X2 = dataframe_info(df_X2)

In [9]:
removed_cols_2 = drop_low_variance(df_X2, df_info_X2, min_var)
print("Удаленные столбцы:", removed_cols_2)

Удаленные столбцы: ['Давление в емкости V-1101', 'Давление на входе в R-1101', 'Давление ВСГ на входе в К-1201', 'Перепад давления реактора R-1101', 'Расход пара перед Е-1103']


In [10]:
df_info_X3 = dataframe_info(df_X3)

In [11]:
removed_cols_3 = drop_low_variance(df_X3, df_info_X3, min_var)
print("Удаленные столбцы:", removed_cols_3)

Удаленные столбцы: ['4PDI_R1201_DACA_PV/Var/MV/Перепад давления реактора R-1201/', '4PI2028_DACA_PV/Var/MV/Давление в емкости V-1201/', '4FI2028_DACA_PV/Var/MV/Возврат ТБ в V-1205 после насосов Р-1203/', '4PIC1016_PIDA_PV/Var/MV/Давление на входе в R-1101/', '4PDI_R1101_DACA_PV/Var/MV/Перепад давления реактора R-1101/', '4PI1015_DACA_PV/Var/MV/Давление ТБ перед Е-1101/', '4PIC2035_PIDA_PV/Var/MV/Давление в емкости V-1202/', '4PIC1035A_PIDA_PV/Var/MV/Давление в емкости V-1101/']
